# Training and Evaluation of GNNs and LLMs
In this notebook, we train the models on the [MovieLens Dataset](https://movielens.org/) after the Pytorch Geometrics Tutorial on [Link Prediction](https://colab.research.google.com/drive/1xpzn1Nvai1ygd_P5Yambc_oe4VBPK_ZT?usp=sharing#scrollTo=vit8xKCiXAue).

First we import all of our dependencies.

The **GNNTrainer** manages and trains a GNN model. Its most important interfaces include
**the constructor**, which defines the GNN architecture and loads the pre-trained GNN model if it is already on the hard disk,
**the training method**, which initializes the training on the GNN model and
**the get_embedding methods**, which represent the inference interface to the GNN model and return the corresponding embeddings in the dimension defined in the constructor for given user movie node pairs.

**The MovieLensLoader** loads and manages the data sets. The most important tasks include **saving and (re)loading and transforming** the data sets.

**PromptEncoderOnlyClassifier** and **VanillaEncoderOnlyClassifier** each manage a **prompt (model) LLM** and a **vanilla (model) LLM**. An EncoderOnlyClassifier (ClassifierBase) provides interfaces for training and testing an LLM model.
PromptEncoder and VanillaEncoder differ from their DataCollectors. DataCollectors change the behavior of the models during training and testing and allow data points to be created at runtime. With the help of these collators, we **create non-existent edges on the fly**.

In [1]:
from gnn import GNNTrainer
from movie_lens_loader import MovieLensLoader
from llm import PromptBertClassifier, VanillaBertClassifier, AddingEmbeddingsBertClassifierBase

from transformers import AutoConfig

We define in advance which **Knowledge Graph Embedding Dimension (KGE_DIMENSION)** the GNN encoder has. We want to determine from which output dimension the GNN encoder can produce embeddings that lead to a significant increase in performance *without exceeding the context length of the LLMs*. In the original tutorial, the KGE_DIMENSION was $64$.

In [2]:
MODEL_NAME = "google/bert_uncased_L-2_H-128_A-2"
MODEL_HIDDEN_SIZE = AutoConfig.from_pretrained(MODEL_NAME).hidden_size
SMALL_KGE_DIMENSION = 4
LARGE_KGE_DIMENSION = MODEL_HIDDEN_SIZE
KGE_DIMENSIONS = [SMALL_KGE_DIMENSION, LARGE_KGE_DIMENSION] # Output Dimension of the GNN Encoder.
EPOCHS = 20
BATCH_SIZE = 256

First we load the MovieLensLoader, which downloads the Movie Lens dataset (https://files.grouplens.org/datasets/movielens/ml-latest-small.zip) and prepares it to be used on GNN and LLM. We also pass the embedding dimensions that we will assume we are training with. First time takes approx. 30 sec.

In [3]:

movie_lens_loader = MovieLensLoader(kge_dimensions = KGE_DIMENSIONS)

In [4]:
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",user: 0[SEP]title: Toy Story (1995)[SEP]genres...,val
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",user: 0[SEP]title: Grumpier Old Men (1995)[SEP...,train
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",user: 0[SEP]title: Heat (1995)[SEP]genres: ['A...,train
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",user: 0[SEP]title: Seven (a.k.a. Se7en) (1995)...,train
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']","user: 0[SEP]title: Usual Suspects, The (1995)[...",test


Next, we initialize the GNN trainers (possible on Cuda), one for each KGE_DIMENSION.
A GNN trainer manages a model and each model consists of an **encoder and classifier** part.

**The encoder** is a parameterized *Grap Convolutional Network (GCN)* with a *2-layer GNN computation graph* and a single *ReLU* activation function in between.

**The classifier** applies the dot-product between source and destination kges to derive edge-level predictions.

In [5]:
gnn_trainer =    GNNTrainer(movie_lens_loader.data, kge_dimension = SMALL_KGE_DIMENSION)
gnn_trainer_large = GNNTrainer(movie_lens_loader.data, hidden_channels=MODEL_HIDDEN_SIZE, kge_dimension=MODEL_HIDDEN_SIZE)

loading pretrained model
Device: 'cpu'
loading pretrained model
Device: 'cpu'


We then train and validate the model on the link prediction task.

If the model is already trained, we can skip this part.
Training the models can take up to 5 minutes.

In [6]:

print(gnn_trainer.kge_dimension)
#gnn_trainer.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
#gnn_trainer.validate_model(movie_lens_loader.gnn_test_data)
print(gnn_trainer_large.kge_dimension)
#gnn_trainer_large.train_model(movie_lens_loader.gnn_train_data, EPOCHS)
#gnn_trainer_large.validate_model(movie_lens_loader.gnn_test_data)


4
128


Next we produce the KGEs for every edge in the dataset. These embeddings can then be used for the LLM on the link-prediction task.

In [7]:
gnn_trainer.get_embeddings(movie_lens_loader)
gnn_trainer_large.get_embeddings(movie_lens_loader)
movie_lens_loader.llm_df.head()

,mappedUserId,mappedMovieId,title,genres,prompt,split,user_embedding_4,movie_embedding_4,user_embedding_128,movie_embedding_128
0,0,0,Toy Story (1995),"['Adventure', 'Animation', 'Children', 'Comedy...",user: 0[SEP]title: Toy Story (1995)[SEP]genres...,val,"[-1.1815502643585205, 1.967491865158081, 1.330...","[-0.4464290738105774, 0.6415067911148071, 1.57...","[0.07740482687950134, -0.7918119430541992, -0....","[0.17317555844783783, -0.26109546422958374, -0..."
1,0,2,Grumpier Old Men (1995),"['Comedy', 'Romance']",user: 0[SEP]title: Grumpier Old Men (1995)[SEP...,train,"[-1.571462631225586, 2.0026934146881104, 1.266...","[0.22539174556732178, 0.17622840404510498, 2.0...","[0.22581057250499725, -0.958337128162384, -0.0...","[0.38612648844718933, 0.47033774852752686, -0...."
2,0,5,Heat (1995),"['Action', 'Crime', 'Thriller']",user: 0[SEP]title: Heat (1995)[SEP]genres: ['A...,train,"[-0.9970571994781494, 1.9963066577911377, 1.26...","[-0.07397547364234924, -0.20919907093048096, 2...","[0.044068194925785065, -0.8816429376602173, -0...","[-0.060755714774131775, 0.32998985052108765, -..."
3,0,43,Seven (a.k.a. Se7en) (1995),"['Mystery', 'Thriller']",user: 0[SEP]title: Seven (a.k.a. Se7en) (1995)...,train,"[-0.9177595973014832, 2.1703622341156006, 1.22...","[-1.090133786201477, 0.09436404705047607, 1.98...","[0.06730379909276962, -1.0015345811843872, -0....","[0.20294731855392456, 0.1033535748720169, -0.3..."
4,0,46,"Usual Suspects, The (1995)","['Crime', 'Mystery', 'Thriller']","user: 0[SEP]title: Usual Suspects, The (1995)[...",test,"[-0.6725887656211853, 2.437504768371582, 1.153...","[-1.158760905265808, 0.5077576637268066, 1.141...","[0.06638426333665848, -0.9026688933372498, -0....","[0.21229048073291779, 0.33715885877609253, -0...."


Next we initialize the vanilla encoder only classifier. This classifier does only use the NLP part of the prompt (no KGE) for predicting if the given link exists.

In [8]:
vanilla_bert_classifier = VanillaBertClassifier(movie_lens_loader.llm_df, batch_size=BATCH_SIZE, model_name=MODEL_NAME)

Next we generate a vanilla llm dataset and tokenize it for training.

In [9]:
dataset_vanilla = movie_lens_loader.generate_vanilla_dataset(vanilla_bert_classifier.tokenize_function)

Next we train the model on the produced dataset. This can be skipped, if already trained ones.

In [10]:
#vanilla_bert_classifier.train_model_on_data(dataset_vanilla, epochs=EPOCHS)

Next we initialize the prompt encoder only classifier. This classifier uses the vanilla prompt and the KGEs for its link prediction.

In [11]:
prompt_bert_classifier = PromptBertClassifier(movie_lens_loader, gnn_trainer.get_embedding, kge_dimension=SMALL_KGE_DIMENSION, batch_size=BATCH_SIZE, model_max_length = 512, model_name=MODEL_NAME, force_recompute=True)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google/bert_uncased_L-2_H-128_A-2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


We also generate a prompt dataset, this time the prompts also include the KGEs.

In [12]:
dataset_prompt = movie_lens_loader.generate_prompt_embedding_dataset(prompt_bert_classifier.tokenize_function, kge_dimension = prompt_bert_classifier.kge_dimension)

We also train the model. This can be skipped if already done ones.

In [13]:
#prompt_bert_classifier.train_model_on_data(dataset_prompt, epochs = EPOCHS)

In [14]:
adding_embedding_bert_only_classifier = AddingEmbeddingsBertClassifierBase(movie_lens_loader, gnn_trainer_large.get_embedding, kge_dimension=MODEL_HIDDEN_SIZE, batch_size=BATCH_SIZE, model_name=MODEL_NAME)
dataset_adding_embedding = movie_lens_loader.generate_adding_embedding_dataset(adding_embedding_bert_only_classifier.tokenizer.sep_token, adding_embedding_bert_only_classifier.tokenizer.pad_token, adding_embedding_bert_only_classifier.tokenize_function, kge_dimension = MODEL_HIDDEN_SIZE)

In [15]:
#adding_embedding_bert_only_classifier.train_model_on_data(dataset_adding_embedding, epochs = EPOCHS)

In [16]:
vanilla_bert_classifier.forward_dataset_and_save_outputs(dataset_vanilla, force_recompute = True)
prompt_bert_classifier.forward_dataset_and_save_outputs(dataset_prompt, force_recompute = True)
adding_embedding_bert_only_classifier.forward_dataset_and_save_outputs(dataset_adding_embedding, force_recompute = True)

BertSdpaSelfAttention is used but `torch.nn.functional.scaled_dot_product_attention` does not support non-absolute `position_embedding_type` or `output_attentions=True` or `head_mask`. Falling back to the manual attention implementation, but specifying the manual implementation will be required from Transformers version v5.0.0 onwards. This warning can be removed using the argument `attn_implementation="eager"` when loading the model.


,user_id,title,genres,labels,split,hidden_states,attentions,graph_embeddings
0,392,gonzo : the life and work of dr. hunter s. tho...,[documentary],0,val,"[[tensor(-0.8975), tensor(1.1486), tensor(-0.3...","[[[tensor(0.1124), tensor(0.1160)], [tensor(0....","[[tensor(0.1376), tensor(-0.9951), tensor(0.34..."
1,245,"very brady sequel, a ( 1996 )",[comedy],0,val,"[[tensor(-0.4546), tensor(0.7260), tensor(-0.3...","[[[tensor(0.1006), tensor(0.0869)], [tensor(0....","[[tensor(-0.0096), tensor(-0.6238), tensor(0.2..."
2,0,dumb & dumber ( dumb and dumber ) ( 1994 ),"[adventure , comedy]",1,val,"[[tensor(0.2223), tensor(0.6985), tensor(-0.40...","[[[tensor(0.1754), tensor(0.1472)], [tensor(0....","[[tensor(0.1465), tensor(-0.9264), tensor(-0.2..."
3,411,death wish 5 : the face of death ( 1994 ),"[action , drama]",0,val,"[[tensor(-0.6065), tensor(0.9631), tensor(-0.4...","[[[tensor(0.0805), tensor(0.0988)], [tensor(0....","[[tensor(-0.1855), tensor(-0.0592), tensor(-0...."
4,513,washington square ( 1997 ),[drama],0,val,"[[tensor(-1.1391), tensor(1.0823), tensor(-0.2...","[[[tensor(0.0626), tensor(0.0691)], [tensor(0....","[[tensor(0.1202), tensor(-0.5565), tensor(0.79..."
...,...,...,...,...,...,...,...,...
187,546,heidi ( 1937 ),"[children , drama]",0,val,"[[tensor(-0.9938), tensor(1.1621), tensor(-0.3...","[[[tensor(0.0075), tensor(0.0085)], [tensor(0....","[[tensor(-0.3378), tensor(-1.2377), tensor(0.7..."
188,3,"princess bride, the ( 1987 )","[action , adventure , comedy , fantasy , romance]",1,val,"[[tensor(-0.2865), tensor(0.9831), tensor(-0.3...","[[[tensor(0.0081), tensor(0.0190)], [tensor(0....","[[tensor(0.2737), tensor(-0.1868), tensor(-0.3..."
189,317,wal - mart : the high cost of low price ( 2005 ),[documentary],0,val,"[[tensor(-1.1251), tensor(1.5771), tensor(-0.4...","[[[tensor(0.0079), tensor(0.0084)], [tensor(0....","[[tensor(0.5799), tensor(0.0635), tensor(-0.10..."
190,319,strange days ( 1995 ),"[action , crime , drama , mystery , sci - fi ,...",0,val,"[[tensor(-0.9166), tensor(0.7996), tensor(-0.4...","[[[tensor(0.0080), tensor(0.0153)], [tensor(0....","[[tensor(-0.0205), tensor(-0.6738), tensor(0.2..."
